# Proyecto Final

## Regresión Lineal univariable

### 1. Cargar Librerías

In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### 2. Cargar Información

In [87]:
data = pd.DataFrame(np.load('proyecto_training_data.npy'))
data=data.set_axis(['SalePrice', 'OverallQual', '1stFlrSF', 'TotRmsAbvGrd', 'YearBuilt', 'LotFrontage'], axis=1)
rowCount = data.shape[0]

#### 2.1 Dividiendo datos para entrenamiento y para validación

In [88]:
trainingDataPercentage = 0.8

IsTrainingData = np.random.choice([0,1], p=[1-trainingDataPercentage, trainingDataPercentage], size=(rowCount))

while(np.sum(IsTrainingData) == np.ceil(rowCount*trainingDataPercentage)):
    IsTrainingData = np.random.choice([0,1], p=[1-trainingDataPercentage, trainingDataPercentage], size=(rowCount))
    
data['training']=IsTrainingData

trainingData = data.loc[data['training'] == 1]
validationData = data.loc[data['training'] == 0]

## 3. Análisis Exploratorio

In [91]:
stats = pd.DataFrame({
    'Variable':pd.Series(['SalePrice', 'OverallQual', '1stFlrSF', 'TotRmsAbvGrd', 'YearBuilt', 'LotFrontage']),
    'Media':pd.Series([np.nanmean(trainingData['SalePrice']), np.nanmean(trainingData['OverallQual']), np.nanmean(trainingData['1stFlrSF']), np.nanmean(trainingData['TotRmsAbvGrd']), np.nanmean(trainingData['YearBuilt']), np.nanmean(trainingData['LotFrontage'])]),
    'Valor Máximo':pd.Series([np.nanmax(trainingData['SalePrice']), np.nanmax(trainingData['OverallQual']), np.nanmax(trainingData['1stFlrSF']), np.nanmax(trainingData['TotRmsAbvGrd']), np.nanmax(trainingData['YearBuilt']), np.nanmax(trainingData['LotFrontage'])]),
    'Valor Mínimo':pd.Series([np.nanmin(trainingData['SalePrice']), np.nanmin(trainingData['OverallQual']), np.nanmin(trainingData['1stFlrSF']), np.nanmin(trainingData['TotRmsAbvGrd']), np.nanmin(trainingData['YearBuilt']), np.nanmin(trainingData['LotFrontage'])]),
    'Rango':pd.Series([np.ptp(trainingData['SalePrice']), np.ptp(trainingData['OverallQual']), np.ptp(trainingData['1stFlrSF']), np.ptp(trainingData['TotRmsAbvGrd']), np.ptp(trainingData['YearBuilt']), np.ptp(trainingData['LotFrontage'])]),
    'Desviación Estándar':pd.Series([np.std(trainingData['SalePrice']), np.std(trainingData['OverallQual']), np.std(trainingData['1stFlrSF']), np.std(trainingData['TotRmsAbvGrd']), np.std(trainingData['YearBuilt']), np.std(trainingData['LotFrontage'])]),
})
stats

,Variable,Media,Valor Máximo,Valor Mínimo,Rango,Desviación Estándar
0,SalePrice,180051.595458,755000.0,34900.0,720100.0,79601.447094
1,OverallQual,6.084945,10.0,1.0,9.0,1.378808
2,1stFlrSF,1154.738436,4692.0,372.0,4320.0,386.382293
3,TotRmsAbvGrd,6.532380,14.0,3.0,11.0,1.645649
4,YearBuilt,1970.582843,2010.0,1880.0,130.0,30.581456
5,LotFrontage,69.954407,313.0,21.0,NaN,24.658497
